In [44]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
%matplotlib inline

algo = "HC"
base_path = "/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/"
pictures_path = f"{base_path}Pictures/"
dataset_path = f"{base_path}Survey+dataset/"
code_path = f"{base_path}Code/"
results_path = f"{base_path}Code/Data/"
HC_base_path = f"{results_path}{algo}/"

import sys
sys.path.append(code_path)
np.set_printoptions(precision=5, suppress=True)

## Plot elbow

In [80]:
import metrics.elbow
import importlib
importlib.reload(metrics.elbow)

<module 'metrics.elbow' from '/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code/metrics/elbow.py'>

In [81]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from scipy.spatial.distance import cdist, pdist

from metrics.elbow import PlotElbow, getElbows

def vdm_prepare_df(df, cat_ix):
  replace_arr = {}
  for col in df.columns[~cat_ix]:
    replace_arr[col] = { value: key for key, value in enumerate(np.sort(df[col].unique()))}
  return df.replace(replace_arr)

# ------- config vars ---------
algo = "HC"
metric="VDM"
link_file_type = ".npy"
wss_file_type = ".npy"
vars_list = [5]
range_min = 2
range_max = 100
postprocessing = False
is_postprocessing = "_fix" if postprocessing else ""

base_path = "/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/"
pictures_path = f"{base_path}Pictures/"
dataset_path = f"{base_path}Survey+dataset/"
results_path = f"{base_path}Code/Data/"
HC_base_path = f"{results_path}{algo}/"

# ------- compute wss/bss --------
linkages = ["single", "complete", "average", "weighted"]#, "ward", "centroid", "median"]
for vars in vars_list:
  print(f"vars: {vars}")

  for link in linkages:
    
    WSS_path = f"{HC_base_path}{metric}{vars}/WSS/WSS_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{wss_file_type}"
    wss = np.load(WSS_path)
    elbows = getElbows(wss, range_min, add_first_k=True)
    # elbows = None
    print(elbows)

    # Warning: Run this part in a normal cell of colab
    save_path = f"{pictures_path}{algo}/Knee_Elbow/{metric}{vars}/"
    ke_file_type = ".html"
    save_path += f"Elbow_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{ke_file_type}"
    PlotElbow(wss,range(range_min, range_max), metric +' Distance'+' (' + link + ')' + ' Elbows:' + str(elbows), save_path)
    print(link, "done")


vars: 5
[2, 4, 6, 14, 19, 24, 28, 33, 49, 86]


/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code/metrics/elbow.py:31: RuntimeWarning:

divide by zero encountered in double_scalars



single done
[2, 4, 6, 8, 16, 19, 25, 37]


complete done
[2, 3, 5, 7, 9, 12, 14, 20, 22, 28, 31, 45]


average done
[2, 4, 6, 8, 11, 15, 21, 23, 42, 44]


weighted done


In [46]:
def getElbowsNotWorking(wss, k_offset, add_first_k=False, elbow_angle=0.01, d_dk_th=0.005):
  wss /= np.max(wss) # normalize
  l = []
  if add_first_k:
    l.append(k_offset)
  for k in range(1, wss.shape[0] - 1):
    d_dk_prev = wss[k] - wss[k-1]
    d_dk_next = wss[k+1] - wss[k]
    print(k + k_offset, d_dk_next - d_dk_prev, sep='\t')

    if (d_dk_next - d_dk_prev) > elbow_angle:
      l.append(k + k_offset)
  return l

In [78]:
def getElbows(wss, k_offset, add_first_k=False, elbow_angle=5, d_dk_th=0.001):
  wss /= np.max(wss) # normalize
  # print(wss)
  l = []
  if add_first_k:
    l.append(k_offset)
  for k in range(1, wss.shape[0] - 1):
    d_dk_prev = wss[k] - wss[k-1]
    d_dk_next = wss[k+1] - wss[k]
    if abs(d_dk_next - d_dk_prev) < 10*d_dk_th:
      d_dk_prev = 0
      d_dk_next = 1.0
    elif abs(d_dk_next) < d_dk_th:
      d_dk_next = -0.0
    # print(k + k_offset, d_dk_prev, d_dk_next, d_dk_prev/d_dk_next, sep='\t')
    if (d_dk_prev/d_dk_next) > elbow_angle:
      l.append(k + k_offset)
  return l

In [57]:
linkages = ["single", "complete", "average", "weighted"]#, "ward", "centroid", "median"]
link = 'single'
WSS_path = f"{HC_base_path}{metric}{vars}/WSS/WSS_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{wss_file_type}"
wss = np.load(WSS_path)
getElbows(wss, range_min, add_first_k=True)

[1.      1.      0.89787 0.89787 0.83158 0.83097 0.83084 0.83054 0.83004
 0.83004 0.83004 0.83004 0.78618 0.78618 0.78618 0.78618 0.78618 0.7385
 0.73681 0.7365  0.7365  0.7365  0.70671 0.70671 0.70671 0.67465 0.63894
 0.63894 0.63894 0.63894 0.63894 0.48798 0.48771 0.48771 0.48771 0.48771
 0.48771 0.48771 0.48771 0.48771 0.48771 0.48771 0.48771 0.48771 0.48771
 0.48771 0.48771 0.36596 0.36531 0.36531 0.36469 0.36466 0.36389 0.36376
 0.36376 0.36376 0.36376 0.36376 0.36376 0.35499 0.35499 0.35336 0.35334
 0.35253 0.35253 0.3513  0.34932 0.34932 0.34929 0.34913 0.34913 0.34894
 0.34878 0.34878 0.3487  0.3487  0.3487  0.3487  0.34601 0.34599 0.34599
 0.34488 0.34414 0.34414 0.29893 0.29893 0.29893 0.29893 0.29893 0.29893
 0.29893 0.29893 0.29893 0.29893 0.29893 0.29893 0.29893 0.29893]
3	0.0	-0.10213426623016575	-0.0
4	-0.10213426623016575	-0.0	inf
5	0.0	-0.06628446905706098	-0.0
6	-0.06628446905706098	-0.0006148506023861966	107.80581298906615
7	0	1.0	0.0
8	0	1.0	0.0
9	0	1.0	0.0
10	0	1.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


[2, 4, 6, 14, 19, 20, 24, 27, 28, 33, 49, 61, 63, 68, 80, 86]

In [ ]:
link = "single"
WSS_path = f"{HC_base_path}{metric}{vars}/WSS/WSS_{metric}_{link}_range({range_min},{range_max}){is_postprocessing}{wss_file_type}"
wss = np.load(WSS_path)
wss.shape

(98,)